In [ ]:
!pip install goose3[all] textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.8 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=0ffb51b183765bb05a4c4ff58be7cec5b0466efc66d0d37b5c7ae57f2e1b501f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import nltk
nltk.download('all')
nltk.download('stopwords')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
import pandas as pd
import numpy as np
from goose3 import Goose
import textstat
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import re
import os

# Data Extraction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def data_extraction():
    data=pd.read_excel("/content/drive/MyDrive/Text_Analysis/Input.xlsx")
    df=pd.DataFrame({"title":[],"text":[]})
    for i in range(len(data)):
        g = Goose({'browser_user_agent': 'Mozilla', 'parser_class':'soup','strict': False})
        article = g.extract(url=data.loc[i].URL)
        title=article.title
        text=article.cleaned_text.replace('\n','')
        df.loc[i]=[title, text]
    final_data=pd.concat([data,df],axis=1)
    return final_data

# Text Preprocessing and StopWords

In [ ]:
def stop_words():
    #loading stopwords
    StopWords=[]
    path=r"/content/drive/MyDrive/Text_Analysis/StopWords"
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        with open(file_path,"r", encoding="latin-1") as f:
            l=f.read().split('\n')
            l=[val for val in l if val!='']
            StopWords+=l
    # cleaning stopwords
    for word in StopWords:
        if " | " in word:
            split_words=re.split(r' \| ', word, flags=re.IGNORECASE)
            StopWords.extend(split_words)
            StopWords.remove(word)
    StopWords=[val for val in list(set(StopWords)) if len(val)<=30]
    return StopWords

In [ ]:
def positive_dict(StopWords):
    positive_wrds=[]
    with open(r"/content/drive/MyDrive/Text_Analysis/MasterDictionary/positive-words.txt","r", encoding="latin-1") as f:
        positive_wrds+=f.read().split('\n')
    positive_wrds=[wrd for wrd in positive_wrds if wrd not in StopWords]
    return positive_wrds

In [ ]:
def negative_dict(StopWords):
    negative_wrds=[]
    with open("/content/drive/MyDrive/Text_Analysis/MasterDictionary/negative-words.txt","r", encoding="latin-1") as f:
        negative_wrds+=f.read().split('\n')
    negative_wrds=[wrd for wrd in negative_wrds if wrd not in StopWords]
    return negative_wrds

# Score

In [ ]:
def sents(text):
  lst_of_sent=[(''.join(char for char in sentence if char not in punctuation)) for sentence in nltk.sent_tokenize(text)]
  return lst_of_sent

def word_count(text):
  sentences=sents(text)
  stop_words = set(stopwords.words("english"))
  wrd_cnt=0
  for sen in sentences:
    wrd_cnt+=len([word.lower() for word in word_tokenize(sen) if word.isalpha() and word.lower() not in stop_words])
  return wrd_cnt

def positive_score(text,positive_wrds):
  lst_of_wrds=[word.lower() for word in word_tokenize(text) if ((word not in punctuation) and (word in positive_wrds))]
  pos_score=len(lst_of_wrds)
  return pos_score

def negative_score(text,negative_wrds):
  lst_of_wrds=[word.lower() for word in word_tokenize(text) if ((word not in punctuation) and (word in negative_wrds))]
  neg_score=len(lst_of_wrds)
  return neg_score

def polarity_score(text,positive_wrds,negative_wrds):
  pos=positive_score(text,positive_wrds)
  neg=negative_score(text,negative_wrds)
  pol_score=(pos-neg)/((pos+neg)+0.000001)
  return round(pol_score,2)

def subjectivity_score(text,positive_wrds,negative_wrds):
  pos=positive_score(text,positive_wrds)
  neg=negative_score(text,negative_wrds)
  subj_score=(pos+neg)/((word_count(text)+0.000001))
  return round(subj_score,2)

def avg_sentence_length(text):
    no_words = len([word.lower() for word in word_tokenize(text) if (word not in punctuation)])
    no_sentences = len(sents(text))
    average_sentence_length = round(no_words/no_sentences,2)
    return average_sentence_length

def avg_word_length(text):
  total_words = len([word.lower() for word in word_tokenize(text) if (word not in punctuation)])
  total_no_of_char=textstat.letter_count(text, ignore_spaces=True)
  avg_word_length=total_no_of_char/total_words
  return round(avg_word_length,2)

def avg_no_of_words_per_sentences(text):
  total_words=len([word.lower() for word in word_tokenize(text) if (word not in punctuation)])
  total_sents=len(sents(text))
  avg_words_per_sentences=total_words/total_sents
  return round(avg_words_per_sentences,2)

def fog_index(text):
  return round(textstat.gunning_fog(text), 2)

def syllable_count(word):
    vow=['a','e','i','o','u']
    if word.endswith("es") or word.endswith("ed"):
        return 0
    else:
        cnt=0
        for chr in word:
            if chr in vow:
                cnt+=1
        return cnt

def syllable_cnt_per_word(text):
    lst_of_wrds=[word.lower() for word in word_tokenize(text) if (word not in punctuation)]
    cnt_syllable_per_wrd=0
    for word in lst_of_wrds:
        cnt_syllable_per_wrd+=syllable_count(word)
    return cnt_syllable_per_wrd

def complex_words_count(text):
    stop_words = set(stopwords.words("english"))
    lst_of_wrds=[word.lower() for word in word_tokenize(text) if (word not in punctuation) and (word not in stop_words)]
    comp_words_set = set()
    for wrd in lst_of_wrds:
        syllable_cnt = syllable_count(wrd)
        if syllable_cnt > 2:
            comp_words_set.add(wrd)
    return len(comp_words_set)

def pct_of_complex_wrds(text):
  no_complex_words=complex_words_count(text)
  no_words=word_count(text)
  pct=no_complex_words/no_words
  return round(pct,2)

def personal_pronouns(text):
    pronouns = ["I", "we", "my", "ours", "us"]
    pattern = r'\b(?:' + '|'.join(pronouns) + r')\b'
    text = re.sub(r'\b(?:us|united states)\b', '', text, flags=re.IGNORECASE)
    matches = re.findall(pattern, text, flags=re.IGNORECASE)
    count = len(matches)
    return count


# Integration

In [ ]:
data = data_extraction()
StopWords = stop_words()
pos_dic = positive_dict(StopWords)
neg_dic = negative_dict(StopWords)

df = pd.DataFrame(index=data.index, columns=[
    'POSITIVE SCORE',
    'NEGATIVE SCORE',
    'POLARITY SCORE',
    'SUBJECTIVITY SCORE',
    'AVG SENTENCE LENGTH',
    'PERCENTAGE OF COMPLEX WORDS',
    'FOG INDEX',
    'AVG NUMBER OF WORDS PER SENTENCE',
    'COMPLEX WORD COUNT',
    'WORD COUNT',
    'SYLLABLE PER WORD',
    'PERSONAL PRONOUNS',
    'AVG WORD LENGTH'
])

for i in range(len(data)):
    txt = data.iloc[i].text
    WordCount = word_count(txt)
    PositiveScore = positive_score(txt, pos_dic)
    NegativeScore = negative_score(txt, neg_dic)
    PolarityScore = polarity_score(txt, pos_dic, neg_dic)
    SubjectiveScore = subjectivity_score(txt, pos_dic, neg_dic)
    AvgSentenceLen = avg_sentence_length(txt)
    AvgWordLen = avg_word_length(txt)
    AvgWordsPerSentences = avg_no_of_words_per_sentences(txt)
    FogIndx = fog_index(txt)
    SyllableCntPerWord = syllable_cnt_per_word(txt)
    ComplxWordCount = complex_words_count(txt)
    PercentageComplexWord = pct_of_complex_wrds(txt)
    PersonalPronoun = personal_pronouns(txt)

    df.iloc[i] = [PositiveScore, NegativeScore, PolarityScore, SubjectiveScore, AvgSentenceLen, PercentageComplexWord,
                  FogIndx, AvgWordsPerSentences, ComplxWordCount, WordCount, SyllableCntPerWord, PersonalPronoun,
                  AvgWordLen]

final_df = pd.concat([data, df], axis=1)
final_df.drop(['title', 'text'], axis=1, inplace=True)
final_df.to_excel(r"/content/drive/MyDrive/Text_Analysis/Output.xlsx")

print("Program executed! Please check the Output excel file in the directory.")


Program executed! Please check the Output excel file in the directory.
